In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
data = pd.read_csv('/content/Tweet Disaster All CSV File v2.0 - Tweet Disaster All CSV File.csv.csv')

<ipython-input-3-ac7e7741c42e>:1: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/Tweet Disaster All CSV File v2.0 - Tweet Disaster All CSV File.csv.csv')


In [ ]:
filtered_data = data[['Tweet Text', 'FloodFlag (4:Non_Flood; 5:Flood)']].dropna()
filtered_data['FloodFlag (4:Non_Flood; 5:Flood)'] = filtered_data[
    'FloodFlag (4:Non_Flood; 5:Flood)'].map({4: 0, 5: 1})

In [ ]:
texts = filtered_data['Tweet Text'].astype(str).values
labels = filtered_data['FloodFlag (4:Non_Flood; 5:Flood)'].astype(int).values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
# Step 2: Tokenization and padding
max_words = 20000  # Maximum number of unique tokens
max_len = 50       # Maximum sequence length for padding


In [ ]:
tokenizer = Tokenizer(num_words=max_words, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding='post', truncating='post')

In [ ]:
vocab_size = min(max_words, len(tokenizer.word_index) + 1)

In [ ]:
# Step 3: Build the BiLSTM model
embedding_dim = 128

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# Step 4: Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping]
)



Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 148ms/step - accuracy: 0.8949 - loss: 0.3888 - val_accuracy: 0.9308 - val_loss: 0.2471
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.9326 - loss: 0.2095 - val_accuracy: 0.9333 - val_loss: 0.2397
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 150ms/step - accuracy: 0.9654 - loss: 0.0918 - val_accuracy: 0.9013 - val_loss: 0.3469
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 127ms/step - accuracy: 0.9950 - loss: 0.0210 - val_accuracy: 0.9207 - val_loss: 0.4624
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 140ms/step - accuracy: 0.9980 - loss: 0.0081 - val_accuracy: 0.9013 - val_loss: 0.5799


In [ ]:
# Step 5: Evaluate the model
loss, accuracy = model.evaluate(X_val_pad, y_val)
print(f"Validation Accuracy: {accuracy:.2f}")

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9298 - loss: 0.2447
Validation Accuracy: 0.93


In [ ]:
# Save the model and tokenizer
model.save("bilstm_flood_model.h5")
with open("tokenizer.json", "w") as f:
    f.write(tokenizer.to_json())


In [ ]:
import numpy as np

# Step 1: Predict probabilities
y_pred_prob = model.predict(X_val_pad)

# Step 2: Convert probabilities to binary class labels
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int).flatten()

# Step 3: Compare with actual labels
correct_predictions = np.sum(y_pred == y_val)
total_predictions = len(y_val)
accuracy = correct_predictions / total_predictions
print(f"Validation Accuracy (manual check): {accuracy:.2f}")

# Step 4: Display a few examples for manual verification
for i in range(10):
    print(f"Tweet: {X_val[i]}")
    print(f"Actual Label: {'Flood-related' if y_val[i] == 1 else 'Non-Flood-related'}")
    print(f"Predicted Label: {'Flood-related' if y_pred[i] == 1 else 'Non-Flood-related'}")
    print("-" * 50)


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Validation Accuracy (manual check): 0.93
Tweet: Kerala is sinking under floods &amp; situation is bad, pray for Kerala @KeralaFloods
Actual Label: Flood-related
Predicted Label: Flood-related
--------------------------------------------------
Tweet: Happy Onam to all :-) Low key celebrations as Kerala limps back to life after being battered and bruised by rains .. Lets stand together and utilise the funds for Onam celebrations for flood victims
Actual Label: Flood-related
Predicted Label: Flood-related
--------------------------------------------------
Tweet: Kudumbashree workers had been active from the initial days of the flood, making packaged meals available to affected people. #KeralaFloods #KeralaFloodsRelief
Actual Label: Flood-related
Predicted Label: Flood-related
--------------------------------------------------
Tweet: There was a warning issued 7 years ago for illegal mining in Kerala. Authorities were warned to stop construction and 